# Building an Internal Chatbot and connecting to external dataset

**Specifically retrieving from a database of legal data on recorded cases**

In [ ]:
#Install packages and dependencies for Colab
# type: ignore

!pip install -U fsspec==2023.6.0 datasets==2.18.0
!pip install datasets s3fs gcsfs --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl sentence-transformers langchain faiss-cpu gradio pymupdf langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Step 1: Load & Fine-Tune Open LLM (QLoRA) -- based on Llama2
# type: ignore

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_id = "openchat/openchat-3.5-0106"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(task_type="CAUSAL_LM", r=8, lora_alpha=16, lora_dropout=0.1, target_modules=["q_proj", "v_proj"])
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
# Step 2: Upload & Embed Documents
# type: ignore

!pip install langchain_community

import pandas as pd # Import pandas for CSV handling
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document # Import Document class
# Removed PyMuPDFLoader as we are handling CSV

from google.colab import files
uploaded_files = files.upload() #uploaded open source legal verbiage dataset from kaggle (https://www.kaggle.com/datasets/amohankumar/legal-text-classification-dataset)

docs = []
for filename, content in uploaded_files.items():
    # Assuming the CSV contains a column with text data, replace 'text_column_name' with the actual column name
    # Also, decode the bytes content to string
    df = pd.read_csv(pd.io.common.BytesIO(content), encoding='utf-8')
    # Convert relevant columns to string and combine them into a single text field per row
    # Replace 'column1', 'column2' with the actual column names you want to include
    df['combined_text'] = df.astype(str).agg(' '.join, axis=1)
    docs.extend([{'page_content': row['combined_text'], 'metadata': {'source': filename, 'row': i}} for i, row in df.iterrows()])


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# Split the combined text from the CSV rows
chunks = splitter.split_documents([Document(page_content=doc['page_content'], metadata=doc['metadata']) for doc in docs])


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embedding_model)
print("Number of vectors in index:", db.index.ntotal) #check if db has been created properly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


Saving legal_text_classification.csv to legal_text_classification.csv


/tmp/ipython-input-4-3373493673.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of vectors in index: 161069


In [ ]:
# type: ignore
from langchain_huggingface import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from transformers import pipeline

# Load your fine-tuned model
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=db.as_retriever(),memory=memory)

Device set to use cuda:0
/tmp/ipython-input-5-3752231665.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
#Check within Jupyter notebook
# type: ignore
from IPython.display import Markdown, display

# Simulate chat history
chat_history = []

# User input
user_input = "What is the purpose of a non-disclosure agreement?"

# Get response from the chatbot
response = qa_chain({
    "question": user_input,
    "chat_history": chat_history
})["answer"]

# Update chat history
chat_history.append((user_input, response))

# Display formatted output
display(Markdown(f"**You:** {user_input}"))
display(Markdown(f"**Bot:** {response}"))

/tmp/ipython-input-6-664195492.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({


**You:** What is the purpose of a non-disclosure agreement?

**Bot:** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

parties. 18 In any event, I consider that in this case there are cogent underlying reasons of principle, similar to those set out in Rush &amp; Tompkins Ltd [1989] 1 AC 1280, to weigh the balance in favour of non-disclosure. This approach protects the important interests in encouraging negotiated settlements of disputes, ensuring that parties in such negotiations are frank and open with each other, and ensuring that parties communicate without apprehension that confidential (and potentially

the purpose of obtaining the agreement of another person under subsection 30(3) to the legal practitioner answering a question or producing a document at an examination before an examiner. (3) If a disclosure is made to a person as permitted by subsection (2) or (4), the following provisions apply: (a) while he or she is a person of a kind to whom a disclosure is so permitted to be made, he or she must not disclose the existence of, or any information about, the summons or notice, or any

that the circumstances of disclosure to the third party condition whether such a party will be enjoined.

is to determine whether there is an agreement and whether it was freely entered into on an informed basis...."

Question: What is the purpose of a non-disclosure agreement?
Helpful Answer: The purpose of a non-disclosure agreement is to protect the important interests in encouraging negotiated settlements of disputes, ensuring that parties in such negotiations are frank and open with each other, and ensuring that parties communicate without apprehension that confidential (and potentially damaging) information will be leaked to third parties.

#### BRIEF

#181878

#### EXPANDED

The purpose of a non-disclosure agreement is to protect the important interests in encouraging negotiated settlements of disputes, ensuring that parties in such negotiations are frank and open with each other, and ensuring that parties communicate without apprehension that confidential (and potentially damaging) information will be leaked to third parties. The agreement may also serve to determine whether there is an agreement and whether it was freely entered into on an informed basis. In this case, the non-disclosure agreement would weigh the balance in favour of non-disclosure in order to protect these interests.

#### OTHER

Non-disclosure agreements

#### KEYWORD(S)

purpose, non-disclosure agreement, confidential, interests, negotiated settlements, disputes, communicating, non-disclosure agreements

##### Purpose of a non-disclosure agreement

The purpose of a non-disclosure agreement (NDA), also known as a confidentiality agreement, is to protect the important interests in encouraging negotiated settlements of disputes. This type of agreement ensures that parties in such negotiations are frank and open with each other, and allows them to communicate without apprehension that confidential (and potentially damaging) information will be leaked to third parties. In some cases, the agreement may also serve to determine whether there is an agreement and whether it was freely entered into on an informed basis.

In the given context, the non-disclosure agreement would weigh the balance in favour of non-disclosure in order to protect the interests mentioned above. The agreement is designed to ensure that parties are transparent and honest with each other during negotiations, and that they can discuss sensitive information without fear of it being shared with unauthorized individuals.

The purpose of obtaining the agreement of another person under subsection 30(3) to the legal practitioner answering a question or producing a document at an examination before an examiner is to ensure that the legal practitioner is authorized to disclose the information, and to protect the interests

In [ ]:
# Step 4: Chatbot UI -- running into some issues on this step
#Script to run the chatbot on UI using streamlit

%%writefile app7_fin.py
import streamlit as st
from langchain_huggingface import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

st.title("📚 Internal Legal Chatbot")
st.write("Ask about uploaded documents or just chat!")

st.session_state.history = []

st.session_state.response = ""
st.session_state.user_input = st.text_input("Ask a question:")

if st.session_state.user_input:
  if st.session_state.user_input.lower().strip() in ["hi", "hello", "hey"]:
    st.session_state.response = "Hey there! 👋 How can I help you today?"
  else:
    x = qa_chain({"question": st.session_state.user_input, "chat_history": st.session_state.history})
    st.session_state.response = x["answer"]

st.session_state.history = st.session_state.history[-6:]
st.session_state.history.append((st.session_state.user_input, st.session_state.response))
st.session_state.user_input = ""

for msg, reply in st.session_state.history:
    st.markdown(f"**You:** {msg}")
    st.markdown(f"**Bot:** {reply}")

Overwriting app7_fin.py


In [ ]:
# type: ignore

!pip install -q streamlit pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("2zoheykhj9ERKHGRJE7WflHheii_7NrTatqqF4AFMCEvzPV5z")  # Replace with your token

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
# type: ignore
import subprocess
import time
from pyngrok import ngrok

# Kill any existing tunnels (optional but helpful during development)
ngrok.kill()
# Start the Streamlit app
subprocess.Popen(["streamlit", "run", "app7_fin.py"])
# Wait a few seconds to ensure the app starts
time.sleep(5)
# Open a public URL via ngrok
public_url = ngrok.connect(addr=8501, proto="http")
print(f"🔗 Your chatbot is live at: {public_url}")

🔗 Your chatbot is live at: NgrokTunnel: "https://272bb72df550.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# type: ignore
from pyngrok import ngrok
ngrok.kill()